In [42]:
import os
from dotenv import load_dotenv

from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools import AIPluginTool

In [43]:
# Load environment variables from .env file
load_dotenv()

# Access the environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [44]:
# Setup environment variables
URL = "http://localhost:3000"

llm = ChatOpenAI(temperature=0)

# AI Agent does best when it only has one available tool
# to engage with URLs
tools = load_tools(["requests_post"], allow_dangerous_tools=True)

# AIPluginTool only fetches and returns the openapi.yaml linked to in /.well-known/ai-plugin.json
# This may need some more work to avoid blowing up LLM context window
tool = AIPluginTool.from_plugin_url(URL + "/.well-known/ai-plugin.json")
tools += [tool]

# Setup an agent to answer the question without further human feedback
agent_chain = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Ask the question, and the agent loop
agent_chain.run(
    "How many lamports does 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625 have?")



> Entering new AgentExecutor chain...
I should use the Solana Labs API to retrieve information about the specified address.
Action: solana
Action Input: 
Observation: Usage Guide: This extension is for exploring Solana blockchain data, such as inspecting what tokens a wallet has or explaining what happened in a transaction. Use it whenever a user asks something that might be related to their Solana account or transaction history.

OpenAPI Spec: {'openapi': '3.0.2', 'info': {'title': 'Solana Labs API', 'description': 'An API for retrieving human-readable information about the Solana blockchain.', 'version': '1.0.0'}, 'servers': [{'url': 'https://chatgpt.solanalabs.com/'}], 'paths': {'/api/handlers/getAssetsByOwner': {'post': {'summary': 'getAssetsByOwner', 'description': 'Accepts Solana publicKey address. Returns Metaplex NFTs owned by the address', 'operationId': 'query_assets_by_owner', 'requestBody': {'content': {'application/json': {'schema': {'$ref': '#/components/schemas/getAcco

'0.042545265 SOL'